In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

In [2]:
def calculate_signal(df, col, target_col):

    _ =df.groupby(col,dropna=False, observed=0).agg(
        count=(target_col,np.size),
        signal=(target_col,"sum")
    )
    _.rename(columns={"signal":target_col},inplace=True)
    _["count_pct"] = _["count"].div(_["count"].sum()).mul(100)
    _[f"{target_col}_pct"] = _[f"{target_col}"].div(_[f"{target_col}"].sum()).mul(100)
    _[f"{target_col}_rate"] = _[f"{target_col}"].div(_["count"]).mul(100)

    return _

In [3]:
def plot_bar_and_line(df, target_col, rotation=0):

    _ = df
    fig, ax = plt.subplots(1,2, figsize=(15,5))
    x_ticks = _.index.astype(str)
    title = f"Histogram & {target_col} of {_.index.name}"

    def func(ax, word):

        ax.bar(x_ticks, _[word])
        ax.set_xticks(x_ticks)
        ax.set_xticklabels(x_ticks, rotation=rotation)
        ax.set_xlabel(_.index.name)
        ax.set_ylabel(word, loc="center")
        ax.set_title(title)

        ax2 = ax.twinx()
        ax2.plot(x_ticks, _[target_col], color='red')
        ax2.set_ylabel(target_col, loc="bottom")

    func(ax[0], "count")
    func(ax[1], "count_pct")

    plt.show()


### calcs

In [4]:
df = pd.read_csv('../toydata/titanic.csv')
print(f"{df.shape}")
df.head(1)

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False


In [41]:
edges = [0,20,40,100]
df["age_binned"] = pd.cut(df["age"], bins=edges, right=False)
edges = [0,10,50,100,600]
df["fare_binned"] = pd.cut(df["fare"], bins=edges, right=False)

df["age_fare_binned"] = df["age_binned"].astype(str) + "_" + df["fare_binned"].astype(str)

_ = calculate_signal(df, target_col = "survived", col = "age_fare_binned")
_["age_binned"] = [i for i,j in _.index.str.split("_")]
_["fare_binned"] = [j for i,j in _.index.str.split("_")]


In [73]:
_.pivot(index = "age_binned", columns="fare_binned", values="count").round(2).style.map(lambda v: 'background-color:darkred;' if v < 20 else None)

fare_binned,"[0, 10)","[10, 50)","[100, 600)","[50, 100)"
age_binned,,,,
"[0, 20)",44,101,11,8
"[20, 40)",158,157,23,49
"[40, 100)",34,82,14,33
nan,100,54,5,18


In [59]:
_2 = _.pivot(index = "age_binned", columns="fare_binned", values="survived_rate").round(2)
_2.style.background_gradient(cmap="RdYlGn")

fare_binned,"[0, 10)","[10, 50)","[100, 600)","[50, 100)"
age_binned,,,,
"[0, 20)",34.090000,49.500000,72.730000,75.000000
"[20, 40)",18.350000,42.680000,78.260000,73.470000
"[40, 100)",8.820000,34.150000,71.430000,60.610000
nan,20.000000,38.890000,60.000000,44.440000
